In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth, riconosci_parcheggio1

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [7]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-2, -25, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-0.8, y=0.5, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [8]:
def camera_callback(image):
    global run
    prediction=riconosci_parcheggio1(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))
    #print(prediction)
    #cv2.imwrite(f'./output5/{image.frame}.png', np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    if prediction > 0.65:
        run = False
camera_rigth.listen(lambda image: camera_callback(image))

In [9]:
def controllo_velocita(control, target_speed_mps, current_speed_mps):
    speed_error = target_speed_mps - current_speed_mps
    if speed_error > 0:
        control.throttle = min(1.0, 0.5 + speed_error * 0.5) 
    elif speed_error < 0:
        control.throttle = 0.0
        control.brake = min(1.0, -speed_error * 0.5) 
    else:
        control.throttle = 0.0
        control.brake = 0.0
    return control

In [10]:
target_speed_mps = 10 / 3.6

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0
target_distance = 2.8
distance_travelled = 0.0

while run:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
while distance_travelled < target_distance:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.1
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
control.brake = 0.0
control.reverse = True
control.steer = 0.8
distance_travelled = 0.0
target_distance = 3.5
while distance_travelled < target_distance:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.1
    print(distance_travelled)
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(1.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()

0.2808930635452271
0.3464971899986268
0.3470968426205219
0.3481899063102902
0.3488522169645876
0.34916173890233043
0.34928771775448697
0.3493415809120052
0.34935057198381403
0.34939392177548145
0.3494156761771592
0.3494356063711166
0.34945440152587254
0.3495128972856037
0.3495214657028555
0.34955242764408473
0.34956103962831553
0.34959263252894746
0.34961316554035876
0.34962670225286274
0.34966108305598026
0.3496933814305521
0.34969795246288415
0.34975300922669705
0.34978473381634106
0.3498001126921736
0.34981759373913524
0.34982124537746134
0.34982324353459265
0.34983779151934874
0.34985043827309714
0.3498546480406731
0.3498585247501979
0.34986329571493124
0.34986961422055185
0.3498732943184224
0.34998227130654425
0.35007678136262266
0.35011929297270405
0.35012185089963177
0.35012237331575313
0.35015375274851984
0.35018859331357827
0.35021202190582706
0.3502200709077443
0.3502287035986229
0.3502554365000834
0.3502735364254478
0.3502821550162481
0.3503366704662768
0.35039881003140194
0

KeyboardInterrupt: 